In [2]:
from langchain_experimental.agents import create_csv_agent
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()
import pandas as pd

In [3]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"))

Here I converted the code into json so it is easy to embed them into the vector database

In [6]:
import csv
import json

csv_file = 'clean_data.csv'
json_file = 'data.json'

data = []
with open(csv_file, 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        data.append(row)

with open(json_file, 'w') as jsonfile:
    json.dump(data, jsonfile, indent=4)

In [4]:
agent1 = create_csv_agent(
            llm, "clean_data.csv", verbose=True)

In [5]:
agent1.run("i have about 100-200$ what is the best hotel i need 2 bedrooms, 2 bathrooms and a TV in the hall")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: The question is asking for the best hotel with specified requirements and a budget of 100-200$. However, the provided dataframe does not contain information about the number of bedrooms, bathrooms, or the presence of a TV in the hall in the hotels. It also does not have detailed price information, only a general 'price_range' column. Therefore, the information provided in the dataframe is not sufficient to answer the question accurately.
Final Answer: The information provided in the dataframe is not sufficient to accurately answer the question about the best hotel within a specified budget and with certain room requirements.

> Finished chain.


'The information provided in the dataframe is not sufficient to accurately answer the question about the best hotel within a specified budget and with certain room requirements.'

In [24]:
# import requests
# import time
# # Define the API endpoint and parameters
# url = "https://api-ares.traversaal.ai/live/predict"
# api_key = "ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a"
# headers = {"x-api-key": api_key, "content-type": "application/json"}
# query = "I want to eat best food san francisco, give me some suggestions"
# api_parameters = {"query": query}
# # Print out the request headers before sending the request
# print("Request Headers:")
# print(headers)
# # Retry the API request if a connection error occurs
# max_retries = 7
# retry_delay = 4
# #api_data={}
# for retry in range(max_retries):
#     try:
#         # Send a request to the API
#         api_response = requests.get(url, json=api_parameters, headers=headers)
#         # Extract the API response data
#         api_data = api_response.json()
#         break
#     except requests.exceptions.ConnectionError:
#         print(f"Connection error occurred. Retrying in {retry_delay} seconds...")
#         time.sleep(retry_delay)
# else:
#     print("Max retries exceeded. Unable to establish a connection to the API.")

# # Create the RAG prompt
# prompt = "I am performing RAG (Retrieve, Augment, Generate) using an API. Here is the context:\n\n"

# print(api_data)

# # # Iterate over the API data and generate responses for each query
# # for item in api_data["data"]:
# #     prompt += f"{item['context']}\n\n"
# #     prompt += f"Query: {item['query']}\n\n"
# #     prompt += "Generate a response using the provided context:\n\n"

# #     # Generate a response using the RAG model
# #     response = llm.invoke(prompt)
# #     # Extract the generated response
# #     generated_response = response

# #     # Print the generated response
# #     print(generated_response)

Request Headers:
{'x-api-key': 'ares_644291f1c734c64a8c586e4b8b069b533390dd46d6fcee0c438f11528b4d8f8a', 'content-type': 'application/json'}
{'message': 'Missing Authentication Token'}


In [10]:
import requests

url = "https://api-ares.traversaal.ai/live/predict"
query = "Taco places in San Francisco"
payload = {
    "query": query,
}
headers = {
  "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
  "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [11]:
response_data = response.json()
print(len(response_data))

1


In [12]:
print(response_data)

{'error': 'Application Internal Server Error. Contact support '}


In [132]:
response_text = response_data['data']['response_text']
web_urls = response_data['data']['web_url']

KeyError: 'data'

In [1]:
print(response_text)

NameError: name 'response_text' is not defined

In [ ]:
print(web_urls)

['https://www.foodandwine.com/travel/europe/italy/pizzas-in-italy', 'https://www.cntraveler.com/galleries/2014-12-01/10-best-pizzerias-in-italy-rome-naples-venice', 'https://www.blueroadstouring.com/blog/uncover-best-pizza-italy', 'https://www.reddit.com/r/askitaly/comments/wvxl8t/best_pizza_in_italyromemilanvenice/', 'https://community.ricksteves.com/travel-forum/italy/best-pizza-in-italy-map-it', 'https://www.timeout.com/news/its-official-these-are-the-best-pizzerias-in-italy-right-now-071323', 'https://www.foodandwine.com/best-pizzeria-world-7106600', 'https://www.enjoytravel.com/en/travel-news/food-and-drink/pizza-in-italy', 'https://theromanguy.com/italy-travel-blog/italy-travel-blog/italy/the-ultimate-guide-to-pizza-in-italy-eat-pizza-like-a-local/', 'https://www.thedailymeal.com/travel/15-best-pizza-places-italy-slideshow/']


In [138]:
prompt = ""
prompt += f"{response_text}\n\n"
prompt += f"Query: Pizza spots in Italy\n\n"
prompt += "Generate a response using the provided context:\n\n"

# Generate a response using the RAG model
response = llm.invoke(prompt)
# Extract the generated response
generated_response = response

# Print the generated response
print(generated_response)

content="There are several notable pizza spots in Italy. The best pizzeria, according to the 50 Top Pizza guide, is Pizzeria I Masanielli located in Caserta, which is known for its high-quality ingredients and gourmet approach to pizza-making. Other significant pizza places include Di Matteo and Pizzeria da Albert in Naples, Ai Marmi in Rome, Mamma Rosa in Marche, Pizzeria da Michele in Naples, Gino Sorbillo in Naples and Milan, Pizzeria Santarpia in Florence, Pizzeria Port'Alba in Naples, Da Remo in Rome, Antico Forno in Venice, Nicola's in Naples, Spontini in Milan, and Pepe in Grani in Caserta. Each of these pizzerias has its own specialty and unique style of pizza-making that contributes to the diverse pizza culture in Italy."
